In [6]:
import pandas as pd
df = pd.read_csv("dataset/DelayedFlights.csv")


In [7]:
df.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop missing values
df = df.dropna()

# Create binary target: 1 = delayed (> 15 mins)
df['Delayed'] = (df['DepDelay'] > 15).astype(int)

# Select features
features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'ArrTime', 'Distance']
target = 'Delayed'

X = df[features]
y = df[target]

In [9]:
from sklearn.model_selection import train_test_split

X_ref, X_curr, y_ref, y_curr = train_test_split(X, y, test_size=0.5, random_state=42)


In [10]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_ref, y_ref)

# Predict on current dataset
y_pred = model.predict(X_curr)


In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

# Target distribution
print(y.value_counts(normalize=True))

# Feature statistics
print(X.describe())

# Correlation heatmap
plt.figure(figsize=(8,6))
sns.heatmap(df[features + [target]].corr(), annot=True, cmap='coolwarm')
plt.title('Feature Correlation Heatmap')
plt.show()

# Histograms
X.hist(figsize=(12, 8))
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
from evidently import Report
from evidently import Dataset, DataDefinition
from evidently.descriptors import Sentiment, TextLength, Contains
from evidently.presets import TextEvals

In [ ]:
from cuml.ensemble import RandomForestClassifier

In [15]:
from evidently import Report
from evidently.presets import DataDriftPreset
from evidently.presets import ClassificationPreset

In [17]:
# Initialize and run the report
class_report = Report(metrics=[ClassificationPreset(probas_threshold=0.5)])
class_report.run(reference_data=X_ref, current_data=X_curr)

# Display the report inline (useful in Jupyter notebooks)
class_report.show(mode='inline')

# Save the report as an HTML file
class_report.save_html("classification_report.html")

ValueError: Cannot use ClassificationPreset without a classification configration

In [19]:
# Initialize and run the report
report = Report(metrics=[ClassificationPreset()])
report.run(reference_data=X_ref, current_data=X_curr)

# Save the report as an HTML file
report.save_html("model_quality_report.html")

ValueError: Cannot use ClassificationPreset without a classification configration

In [ ]:
# Combine features and target for both datasets
ref_data = X_ref.copy()
ref_data['Delayed'] = y_ref

curr_data = X_curr.copy()
curr_data['Delayed'] = y_curr

# Initialize and run the data drift report
drift_report = Report(metrics=[DataDriftPreset()])
drift_report.run(reference_data=ref_data, current_data=curr_data)

# Save the report as an HTML file
drift_report.save_html("data_drift_report.html")

In [ ]:
import joblib

# Save the model to a file
joblib.dump(model, "random_forest_model.pkl")

# Load the model from the file
# model = joblib.load("random_forest_model.pkl")